In [2]:
import os
os.getcwd()

'd:\\Coding\\War of the ring\\War-of-the-Ring\\notebook'

In [1]:
root = "../src/assets/map/"

In [20]:
import pygame
from PIL import Image
from dataclasses import dataclass

In [11]:
# pygame window dimensions
WINDOW_WIDTH = 1280
WINDOW_HEIGHT = 720

# map dimension
map_image = Image.open(root + "map_cut.png")
map_mask = Image.open(root + "map_mask.png")
MAP_WIDTH = map_image.width
MAP_HEIGHT = map_image.height

In [12]:
print(f"map width = {MAP_WIDTH}")
print(f"map height = {MAP_HEIGHT}")

map width = 6854
map height = 4708


In [13]:
mapheight_fit_factor = WINDOW_HEIGHT/MAP_HEIGHT

In [31]:
class TextClass:
    def __init__(self, text_content : str , font_loc : str,
                 font_size : int, font_color : tuple[int, int, int],
                 rect_center : tuple[int, int]):
        self.text_content = text_content
        self.font_loc = font_loc
        self.font_size = font_size
        self.font_color = font_color
        self.font = pygame.font.SysFont(self.font_loc, self.font_size)
        self.text = self.font.render(self.text_content, True, self.font_color)
        self.text_rect = self.text.get_rect()
        self.text_rect.center = rect_center

    def draw_text(self, screen : pygame.Surface):
        screen.blit(self.text, self.text_rect)

class Quit_Box:
    def __init__(self, text_values : dict, 
                 x : int = 0, y : int = 0,
                 width : int = 100, height : int = 50,
                 ):
        # box
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        # text
        text_values['rect_center'] = (self.x + self.width/2, self.y + self.height/2)
        self.text = TextClass(**text_values)

    def draw(self, screen : pygame.Surface) -> None:
        # draw quit game box at top left
        pygame.draw.rect(screen, (255, 255, 255), pygame.Rect(self.x, self.y, self.width, self.height))
        self.text.draw_text(screen)

    def is_inside(self, mouse_x : int, mouse_y : int) -> bool:
        if mouse_x >= self.x and mouse_x <= self.x + self.width\
        and mouse_y >= self.y and mouse_y <= self.y + self.height:
            return True
        return False

In [38]:
# Initialize Pygame
pygame.init()

# ui elements
text_values = {
    "text_content" : "Quit",
    "font_loc" : root + "font.ttf",
    "font_size" : 40, "font_color" : (0, 0, 0),
    "rect_center" : (0, 0)
}
quit_box = Quit_Box(text_values=text_values)

screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption("War of the Ring")
clock = pygame.time.Clock()
running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
            mouse_x, mouse_y = pygame.mouse.get_pos()
            if quit_box.is_inside(mouse_x, mouse_y):
                running = False

    screen.fill((0, 0, 0))

    # render the frame
    # map

    # ui
    quit_box.draw(screen)

    pygame.display.flip()
    clock.tick(60)

pygame.quit()